<a href="https://colab.research.google.com/github/gujjuri/CalenderApp/blob/main/Ver3_stocks_Universe_Ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip -q install kiteconnect scipy gspread gspread_dataframe gspread-formatting --upgrade

import time, math, re
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone
from IPython.display import display
from scipy.stats import norm
from kiteconnect import KiteConnect

# Google Sheets
import datetime as dt
import gspread
from gspread_dataframe import set_with_dataframe
from google.colab import auth
from google.auth import default
from gspread_formatting import format_cell_ranges, CellFormat, TextFormat, Color, set_frozen

pd.set_option("display.width", 200)
pd.set_option("display.max_columns", 200)


In [11]:
# ==== API (fill yours) ====

API_KEY    = "f88qa6qenuaf942b"
API_SECRET = "oxnggitrcelnwjnzs1k9rav9lgp3yr3q"

# ==== Your hardcoded list WITH .NS (we'll strip to symbols) ====
STOCK_TICKERS = [
    "3MINDIA.NS","AARTIPHARM.NS","APOLLOHOSP.NS","AVANTEL.NS","BAJFINANCE.NS",
    "BEL.NS","BHARTIARTL.NS","BSE.NS","CARTRADE.NS","CHOICEIN.NS",
    "COFORGE.NS","CUMMINSIND.NS","DIVISLAB.NS","DIXON.NS","ECLERX.NS",
    "EICHERMOT.NS","ELECON.NS","ENRIN.NS","ETERNAL.NS","FINEORG.NS",
    "GOLDBEES.NS","HAL.NS","HAWKINCOOK.NS","HDBFS.NS","HDFCBANK.NS",
    "HDFCLIFE.NS","HEROMOTOCO.NS","HNGSNGBEES.NS","ICICIBANK.NS","ICICIGI.NS",
    "INDIGO.NS","ITC.NS","JIOFIN.NS","KIMS.NS","KPIGREEN.NS",
    "LAURUSLABS.NS","LT.NS","M&M.NS","MAHSCOOTER.NS","MARICO.NS",
    "MARUTI.NS","MAXHEALTH.NS","MCX.NS","METROPOLIS.NS","MFSL.NS",
    "MODEFENCE.NS","MUTHOOTFIN.NS","NATCOPHARM.NS","NAZARA.NS","NESTLEIND.NS",
    "NH.NS","NIFTYBEES.NS","NIVABUPA.NS","NPST.NS","OLECTRA.NS",
    "PATANJALI.NS","PAYTM.NS","POONAWALLA.NS","POWERINDIA.NS","RELIANCE.NS",
    "SBILIFE.NS","SCHAEFFLER.NS","SHAILY.NS","SHILCTECH.NS","SHREECEM.NS",
    "SHRIPISTON.NS","SHRIRAMFIN.NS","SILVERBEES.NS","STAR.NS","SUNDARMHLD.NS",
    "SUPREMEIND.NS","TANLA.NS","TATACOMM.NS","TATAPOWER.NS","TATASTEEL.NS",
    "TFCILTD.NS","TVSMOTOR.NS","UPL.NS","V2RETAIL.NS","VBL.NS",
    "VISHNU.NS","VMM.NS","WAAREEENER.NS","WAAREERTL.NS"
]

# ==== Universe / selection sizes ====
TARGET_COUNT   = 50   # Core allocation size
BUFFER_COUNT   = 10   # Extra list (keep if held; no new buys)
CAL_LOOKBACK_DAYS = 420  # ~14 months

# ==== Benchmark ====
BENCHMARK_NAME = "NIFTY 50"
NIFTY50_FALLBACK_TOKEN = 256265
BENCH_LABEL = "NIFTY50"

# ==== Risk/metrics windows and weights ====
RISK_FREE_ANNUAL = 0.07
WINDOWS = {"1M": 21, "3M": 63, "1Y": 252}

WINDOW_WEIGHTS       = {"1M": 0.20, "3M": 0.30, "1Y": 0.50}
STABILITY_Z_WEIGHT   = 0.10
STABILITY_RANK_WEIGHT= 0.10
BETA_PENALTY_WEIGHT  = 0.10
DRAWDOWN_PENALTY_WEIGHT = 0.10
BASE_SCORE_WEIGHT = 1.00 - (STABILITY_Z_WEIGHT + STABILITY_RANK_WEIGHT + BETA_PENALTY_WEIGHT + DRAWDOWN_PENALTY_WEIGHT)
CAP_Z = 3.0

# ==== Allocation ====
TOTAL_CAPITAL   = 35_00_000  # ₹
CASH_BUFFER_PCT = 0.01       # keep 1% cash
CORE_FLOOR_PCT  = 0.005      # 0.50%
CORE_CAP_PCT    = 0.06       # 6%

# ==== Filters ====
ETF_REGEX = re.compile(r'(?:ETF$|BEES$|GOLD|SILVERBEES|NIFTYBEES|HNGSNGBEES)', re.IGNORECASE)


In [12]:
# --- Login (request_token flow) ---
kite = KiteConnect(api_key=API_KEY)
print("1) Open & login:", kite.login_url())  # After approving, copy request_token from redirect URL.  (Docs)
REQUEST_TOKEN = input("\n2) Paste request_token here: ").strip()
session_data = kite.generate_session(REQUEST_TOKEN, api_secret=API_SECRET)
kite.set_access_token(session_data["access_token"])
print("✅ Kite session established.")
# Login flow + Python client methods per docs.  :contentReference[oaicite:1]{index=1}

# --- Instruments dump (NSE) ---
inst_nse = pd.DataFrame(kite.instruments("NSE"))
inst_nse["tradingsymbol"] = inst_nse["tradingsymbol"].astype(str)
inst_nse["name"] = inst_nse["name"].astype(str)
inst_nse["instrument_type"] = inst_nse["instrument_type"].astype(str)

# --- Resolve NIFTY50 benchmark token robustly ---
bench_row = inst_nse[
    (inst_nse["instrument_type"].str.upper() == "INDICES") &
    (
        (inst_nse["name"].str.upper() == BENCHMARK_NAME.upper()) |
        (inst_nse["tradingsymbol"].str.upper() == BENCHMARK_NAME.upper()) |
        (inst_nse["name"].str.upper().str.contains(BENCHMARK_NAME.upper(), regex=False)) |
        (inst_nse["tradingsymbol"].str.upper().str.contains(BENCHMARK_NAME.upper(), regex=False))
    )
]
bench_token = int(bench_row.iloc[0]["instrument_token"]) if not bench_row.empty else NIFTY50_FALLBACK_TOKEN
print(f"Benchmark token: {bench_token}")

# --- Helpers ---
def norm_sym(s: str) -> str:
    return str(s).upper().replace(".NS","").strip()

def robust_z(series: pd.Series, cap=CAP_Z) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    med = s.median()
    mad = (s - med).abs().median()
    z = (s - med) / (1e-9 if pd.isna(mad) or mad == 0 else (mad/0.6745))
    return z.clip(-cap, cap)

def sortino_annual(excess_df_window: pd.DataFrame) -> pd.Series:
    neg = excess_df_window.clip(upper=0)
    d_std = neg.std(ddof=0, skipna=True)
    mean_ex = excess_df_window.mean(skipna=True)
    return (mean_ex * np.sqrt(252)) / d_std.replace(0, np.nan)

def tier_from_sharpe(s: float) -> str:
    if pd.isna(s):   return "NA"
    if s > 2.0:      return "Excellent"
    if s >= 1.0:     return "Good"
    if s >= 0.5:     return "Average"
    if s >= 0.0:     return "Poor"
    return "Negative"

def last_n(df: pd.DataFrame, n: int) -> pd.DataFrame:
    return df.iloc[-n:] if len(df) >= n else df

def max_drawdown_from_prices(price_df: pd.DataFrame, min_points=60) -> pd.Series:
    out = {}
    for t in price_df.columns:
        s = price_df[t].dropna()
        if len(s) < min_points:
            out[t] = np.nan
            continue
        wealth = s / s.iloc[0]
        dd = wealth / wealth.cummax() - 1.0
        out[t] = dd.min()
    return pd.Series(out, name="MaxDD")

def beta_vs_market(ret_df: pd.DataFrame, market_col: str, min_points=60) -> pd.Series:
    out = {}
    if market_col not in ret_df.columns:
        return pd.Series({c: np.nan for c in ret_df.columns if c != market_col}, name="Beta_1Y")
    m = ret_df[market_col]
    for t in ret_df.columns:
        if t == market_col: continue
        pair = pd.concat([m, ret_df[t]], axis=1).dropna()
        if len(pair) < min_points or pair[market_col].var(ddof=0) == 0:
            out[t] = np.nan
        else:
            cov = pair.cov(ddof=0).iloc[0,1]
            out[t] = cov / pair[market_col].var(ddof=0)
    return pd.Series(out, name="Beta_1Y")

def fetch_daily_history(kite, token: int, start_dt: datetime, end_dt: datetime, interval="day", sleep_s=0.25):
    """Daily close series (tz-naive Asia/Kolkata) via historical_data()."""
    try:
        data = kite.historical_data(token, start_dt, end_dt, interval=interval, continuous=False, oi=False)
        time.sleep(sleep_s)
        if not data:
            return pd.Series(dtype=float)
        df = pd.DataFrame(data)
        s = df.set_index("date")["close"].astype(float)
        s.index = pd.to_datetime(s.index).tz_convert("Asia/Kolkata").tz_localize(None)
        return s.sort_index()
    except Exception as e:
        print(f"Historical fetch failed for token {token}: {e}")
        return pd.Series(dtype=float)
# Historical candles API reference. :contentReference[oaicite:2]{index=2}


1) Open & login: https://kite.zerodha.com/connect/login?api_key=f88qa6qenuaf942b&v=3

2) Paste request_token here: n97UgFtiQuAr9heJ24c41eZ3nIilsuIG
✅ Kite session established.
Benchmark token: 256265


In [13]:
# ===== Strip ".NS" to pure NSE symbols =====
HARDLIST = [norm_sym(s) for s in STOCK_TICKERS]
print("Hardlist count:", len(HARDLIST))

# ===== Permanent exclusion list =====
EXCLUDED_ALWAYS = ["MODEFENCE", "AVANTEL"]  # <--- Add more here if needed
EXCLUDED_ALWAYS = {norm_sym(s) for s in EXCLUDED_ALWAYS}

# ===== Manual exclusions UI (optional extra exclusions) =====
try:
    import ipywidgets as widgets
    from IPython.display import display

    excl_text = widgets.Text(
        placeholder='Extra exclusions e.g., ITC, HDFCBANK',
        description='Exclude:',
        layout=widgets.Layout(width='70%')
    )
    apply_btn = widgets.Button(description='Apply Exclusions')
    ui_out = widgets.Output()

    def _clicked(_):
        with ui_out:
            ui_out.clear_output()
            print("Captured exclusions:", excl_text.value)

    apply_btn.on_click(_clicked)
    display(widgets.HBox([excl_text, apply_btn]), ui_out)
    _exclusion_input_widget = excl_text
except Exception:
    _exclusion_input_widget = None  # fallback to input()

def _parse_exclusions(raw: str) -> set:
    if not raw:
        return set()
    toks = re.split(r'[,\s]+', str(raw))
    toks = [norm_sym(t) for t in toks if t.strip()]
    return set(toks)

# ===== Holdings (Kite Portfolio → symbols) =====
try:
    k_hold = pd.DataFrame(kite.holdings())
    if not k_hold.empty and "tradingsymbol" in k_hold.columns:
        holdings_syms = (
            k_hold["tradingsymbol"]
            .astype(str)
            .map(norm_sym)
            .dropna()
            .unique()
            .tolist()
        )
    else:
        holdings_syms = []
    print(f"Loaded {len(holdings_syms)} holdings from Kite.")
except Exception as _:
    holdings_syms = []
    print("Holdings not available; continuing with hardlist only.")

# ---- Get manual exclusions ----
if _exclusion_input_widget is not None:
    raw_excl = _exclusion_input_widget.value
else:
    try:
        raw_excl = input("Optional extra exclusions (comma-separated): ").strip()
    except Exception:
        raw_excl = ""

manual_exclusions = _parse_exclusions(raw_excl)

# ---- Merge permanent + manual exclusions ----
excl_syms = EXCLUDED_ALWAYS | manual_exclusions
if excl_syms:
    print("Excluding symbols from ALL sources:", sorted(excl_syms))
    HARDLIST = [s for s in HARDLIST if s not in excl_syms]
    holdings_syms = [s for s in holdings_syms if s not in excl_syms]

# ===== Universe = HARDLIST ∪ holdings =====
union_syms = sorted(set(HARDLIST) | set(holdings_syms))
if not union_syms:
    raise RuntimeError("Universe is empty. Check HARDLIST, holdings, or exclusions.")

# ===== Resolve instrument tokens for our universe (robust) =====
if "tradingsymbol" not in inst_nse.columns:
    raise RuntimeError("Instruments dump missing 'tradingsymbol' column.")

inst_idx = inst_nse.set_index("tradingsymbol", drop=False)

rows, missing = [], []
for s in union_syms:
    if s in inst_idx.index:
        r = inst_idx.loc[s]
        if isinstance(r, pd.DataFrame):
            if "instrument_type" in r.columns and (r["instrument_type"].str.upper() == "EQ").any():
                r = r[r["instrument_type"].str.upper() == "EQ"].iloc[0]
            else:
                r = r.iloc[0]
        rows.append(r)
    else:
        missing.append(s)

if missing:
    print("Missing from instruments (skipped):", missing)

sym_rows = pd.DataFrame(rows)

if sym_rows.empty:
    raise RuntimeError("No symbols resolved from instruments. Cannot proceed with download.")

if "tradingsymbol" not in sym_rows.columns or "instrument_token" not in sym_rows.columns:
    raise RuntimeError("Resolved instrument rows missing required columns ('tradingsymbol', 'instrument_token').")

sym_rows = (
    sym_rows
    .drop_duplicates(subset=["tradingsymbol"])
    .rename(columns={"tradingsymbol": "Ticker"})
    .reset_index(drop=True)
)

# ===== Download daily close for benchmark + universe =====
end = datetime.now(timezone.utc)
start = end - timedelta(days=CAL_LOOKBACK_DAYS)

prices = pd.DataFrame()

bench_close = fetch_daily_history(kite, bench_token, start, end, interval="day")
if bench_close.empty:
    raise RuntimeError("Benchmark history fetch failed.")
prices[BENCH_LABEL] = bench_close

kept_symbols = []
for sym, token in sym_rows[["Ticker", "instrument_token"]].itertuples(index=False):
    try:
        s = fetch_daily_history(kite, int(token), start, end, interval="day")
    except Exception as e:
        print(f"Fetch failed for {sym} (token {token}): {e}")
        continue

    if s.empty:
        print(f"Skipping {sym}: no data.")
        continue

    prices[sym] = s
    kept_symbols.append(sym)

prices = prices.sort_index()
print(f"Daily history downloaded for {len(kept_symbols)} symbols.")


Hardlist count: 84


Output()

Loaded 84 holdings from Kite.
Excluding symbols from ALL sources: ['AVANTEL', 'MODEFENCE']
Missing from instruments (skipped): ['HAWKINCOOK', 'SHILCTECH']
Daily history downloaded for 80 symbols.


In [14]:
# Returns & excess
returns = prices.pct_change(fill_method=None)
rf_daily = (1 + RISK_FREE_ANNUAL)**(1/252) - 1
excess = returns.subtract(rf_daily)

ret_slices = {k: last_n(returns, n) for k, n in WINDOWS.items()}
exc_slices = {k: last_n(excess,  n) for k, n in WINDOWS.items()}

metrics = {}
for label in WINDOWS:
    re = ret_slices[label]
    ex = exc_slices[label]
    std_ret  = re.std(ddof=0, skipna=True)
    mean_ex  = ex.mean(skipna=True)
    sharpe   = (mean_ex / std_ret.replace(0, np.nan)) * np.sqrt(252)
    ann_ret  = re.mean(skipna=True) * 252
    ann_vol  = std_ret * np.sqrt(252)
    sortino  = sortino_annual(ex)
    metrics[label] = pd.DataFrame({
        "AnnReturn_%": ann_ret * 100,
        "AnnVol_%": ann_vol * 100,
        "Sharpe": sharpe,
        "Sortino": sortino
    })

# NIFTY ref
nifty_ref = (pd.DataFrame({
    w: metrics[w].loc[BENCH_LABEL, ["Sharpe","AnnReturn_%","AnnVol_%","Sortino"]]
    for w in WINDOWS
}).T.rename_axis("Window").reset_index())

# Long→wide
long_frames = []
for label in WINDOWS:
    df = metrics[label].copy()
    df["Ticker"] = df.index
    df["Window"] = label
    long_frames.append(df.reset_index(drop=True))
long_df = pd.concat(long_frames, ignore_index=True)

wide = long_df.pivot(index="Ticker", columns="Window",
                     values=["Sharpe","AnnReturn_%","AnnVol_%","Sortino"])
wide.columns = [f"{m}_{w}" for m, w in wide.columns]
wide = wide.reset_index()

# Beat + Tiers + Avg Sharpe
for w in WINDOWS:
    bench_val = nifty_ref.loc[nifty_ref["Window"]==w, "Sharpe"].iloc[0]
    wide[f"BeatNifty_{w}"] = (wide.get(f"Sharpe_{w}") > bench_val)
    wide[f"Tier_{w}"] = wide.get(f"Sharpe_{w}").apply(tier_from_sharpe)

wide["Avg_Sharpe"] = wide[[c for c in wide.columns if c.startswith("Sharpe_")]].mean(axis=1, skipna=True)

# Robust z / window scores
sh_long = long_df[["Ticker","Window","Sharpe"]].copy()
sh_long = sh_long[sh_long["Ticker"] != BENCH_LABEL]

z_parts = []
for w in WINDOWS:
    s = sh_long.loc[sh_long["Window"]==w].set_index("Ticker")["Sharpe"]
    z = robust_z(s, cap=CAP_Z)
    score_ser = pd.Series(norm.cdf(z.to_numpy()) * 100.0, index=z.index, name=f"Score_{w}")
    tmp = pd.concat([z.rename(f"z_{w}"), score_ser], axis=1).reset_index().rename(columns={"index":"Ticker"})
    z_parts.append(tmp)

z_score = z_parts[0]
for t in z_parts[1:]:
    z_score = z_score.merge(t, on="Ticker", how="outer")

# Drawdown & Beta
mdd_series = max_drawdown_from_prices(prices[[c for c in prices.columns if c != BENCH_LABEL]], min_points=60)
dd_df = mdd_series.rename("MaxDD").reset_index().rename(columns={"index":"Ticker"})
dd_df["Drawdown_Score"] = (100 * (1 + dd_df["MaxDD"].clip(-0.5, 0) / 0.5)).clip(0, 100)

ret_1y = ret_slices["1Y"]
beta_series = beta_vs_market(ret_1y[[c for c in ret_1y.columns if c in kept_symbols] + [BENCH_LABEL]], market_col=BENCH_LABEL, min_points=60)
beta_df = beta_series.rename("Beta_1Y").reset_index().rename(columns={"index":"Ticker"})
beta_df["BetaPenalty_Score"] = beta_df["Beta_1Y"].apply(lambda b: 100 - 30*max(0.0, b-1) if pd.notna(b) else np.nan).clip(0, 100)

# Stability: z-dispersion + rank-persistence
z_cols = [c for c in z_score.columns if c.startswith("z_")]
z_score["Z_Dispersion"] = z_score[z_cols].std(axis=1, ddof=0)
z_score["StabilityZ_Score"] = (100 - 20 * z_score["Z_Dispersion"]).clip(0, 100)

def rank_series(window_label: str) -> pd.Series:
    s = long_df[long_df["Window"]==window_label].set_index("Ticker")["Sharpe"]
    s = s.drop(index=BENCH_LABEL, errors="ignore")
    return s.rank(ascending=False, method="min")

r1y = rank_series("1Y")
r3m = rank_series("3M")
if len(r1y) and len(r3m):
    max_rank = max(r1y.max(), r3m.max())
    rank_gap = (r1y - r3m).abs()
    rank_persist = (1 - (rank_gap / max(1, (max_rank - 1))).fillna(1)).clip(0,1) * 100
    rank_persist_df = rank_persist.rename("RankPersist_Score").reset_index().rename(columns={"index":"Ticker"})
else:
    rank_persist_df = pd.DataFrame({"Ticker": wide["Ticker"], "RankPersist_Score": 50})

# Merge → full
full = (wide
        .merge(z_score, on="Ticker", how="left")
        .merge(rank_persist_df, on="Ticker", how="left")
        .merge(beta_df, on="Ticker", how="left")
        .merge(dd_df, on="Ticker", how="left"))

# Final score & rank
def weighted_window_score(row):
    s, wsum = 0.0, 0.0
    for w, wt in WINDOW_WEIGHTS.items():
        v = row.get(f"Score_{w}")
        if pd.notna(v): s += wt * v; wsum += wt
    return s / wsum if wsum > 0 else np.nan

full["Weighted_Score"] = full.apply(weighted_window_score, axis=1)
full["StabilityZ_Score"] = full["StabilityZ_Score"].fillna(50)
full["RankPersist_Score"] = full["RankPersist_Score"].fillna(50)
full["BetaPenalty_Score"] = full["BetaPenalty_Score"].fillna(70)
full["Drawdown_Score"] = full["Drawdown_Score"].fillna(60)

full["Final_Score"] = (
    BASE_SCORE_WEIGHT      * full["Weighted_Score"] +
    STABILITY_Z_WEIGHT     * full["StabilityZ_Score"] +
    STABILITY_RANK_WEIGHT  * full["RankPersist_Score"] +
    BETA_PENALTY_WEIGHT    * full["BetaPenalty_Score"] +
    DRAWDOWN_PENALTY_WEIGHT* full["Drawdown_Score"]
)
full["Final_Rank"] = full["Final_Score"].rank(ascending=False, method="min").astype("Int64")
full["Overall_Tier"] = full["Avg_Sharpe"].apply(tier_from_sharpe)

print("✅ Ranking table `full` ready:", len(full), "rows")


✅ Ranking table `full` ready: 81 rows


In [15]:
held_norm = set(holdings_syms)
union_syms = set(HARDLIST) | held_norm

cand = full.dropna(subset=["Ticker","Final_Rank"]).copy()
cand["NormTicker"] = cand["Ticker"].map(norm_sym)
cand["Final_Rank"] = cand["Final_Rank"].astype("Int64")
cand = cand[cand["NormTicker"].isin(union_syms)]
cand = cand.sort_values(["Final_Rank","Final_Score"], ascending=[True, False]).drop_duplicates("NormTicker")

core   = cand.head(TARGET_COUNT).copy()
buffer = cand.iloc[TARGET_COUNT:TARGET_COUNT+BUFFER_COUNT].copy()

CORE_TICKERS   = core["Ticker"].tolist()
CORE_SYMS      = set(core["NormTicker"].tolist())
BUFFER_TICKERS = buffer["Ticker"].tolist()
BUFFER_SYMS    = set(buffer["NormTicker"].tolist())

NO_SELL_SYMS = CORE_SYMS | (BUFFER_SYMS & held_norm)   # don't sell these

print(f"Core={len(CORE_SYMS)}, Buffer={len(BUFFER_SYMS)}, Held∩Buffer={len(BUFFER_SYMS & held_norm)}")


Core=50, Buffer=10, Held∩Buffer=10


In [16]:
# Preconditions
if prices.empty:
    raise RuntimeError("No prices downloaded.")

ranked = full.copy()
ranked["NormTicker"] = ranked["Ticker"].map(norm_sym)

# Core-only for allocation
before = len(ranked)
ranked = ranked[ranked["Ticker"].isin(set(CORE_TICKERS))].copy()
after  = len(ranked)
print(f"Allocator filtered to CORE: {after}/{before} names")

# Enrich with instrument info (for optional filters)
info_cols = ["tradingsymbol","instrument_type","lot_size","tick_size","instrument_token","exchange_token","name","segment","exchange"]
info_df = inst_nse[info_cols].drop_duplicates("tradingsymbol").rename(columns={"tradingsymbol":"Ticker"})
ranked = ranked.merge(info_df, on="Ticker", how="left")

# Exclude ETFs
ranked = ranked[~ranked["Ticker"].astype(str).str.contains(ETF_REGEX, na=False)].copy()

# Last close
last_close_all = prices.iloc[-1].astype(float)
ranked["Last_Close"] = ranked["Ticker"].map(last_close_all).astype(float)
ranked = ranked.dropna(subset=["Last_Close"])

# Raw score → weights
def safe_beta(b):
    if pd.isna(b): return 1.0
    return max(float(b), 0.5)

def dd_penalty(maxdd):
    if pd.isna(maxdd): return 1.0
    m = min(0.0, float(maxdd))
    return 1.0 / (1.0 - m)

LAMBDA_DECAY = 0.06
GAMMA_VOL    = 0.50
DELTA_BETA   = 0.35
ETA_DD       = 0.25

def raw_score(row):
    r   = float(row["Final_Rank"])
    vol = row.get("AnnVol_%_1Y", np.nan)
    vol_frac = (float(vol)/100.0) if pd.notna(vol) and float(vol) > 0 else np.nan
    beta = safe_beta(row.get("Beta_1Y"))
    mdd  = row.get("MaxDD")

    rank_term = math.exp(-LAMBDA_DECAY*(r-1))
    vol_term  = (1.0/vol_frac)**GAMMA_VOL if pd.notna(vol_frac) and vol_frac>0 else 1.0
    beta_term = (1.0/beta)**DELTA_BETA if beta>0 else 1.0
    dd_term   = (dd_penalty(mdd))**ETA_DD

    return rank_term * vol_term * beta_term * dd_term

ranked["Raw"] = ranked.apply(raw_score, axis=1)
ranked = ranked[np.isfinite(ranked["Raw"]) & (ranked["Raw"]>0)].copy()

ranked["Floor"] = CORE_FLOOR_PCT
ranked["Cap"]   = CORE_CAP_PCT

TARGET_SUM = 1.0 - CASH_BUFFER_PCT
w = ranked["Raw"] / ranked["Raw"].sum() * TARGET_SUM
floors = ranked["Floor"].astype(float).clip(lower=0.0)
caps   = ranked["Cap"].astype(float).clip(lower=0.0)
basis  = ranked["Raw"].copy()

def project_with_pos_caps(w, floors, caps, total, basis, iters=12):
    w = w.clip(lower=0.0).copy()
    if w.sum() == 0:
        w[:] = total / len(w); return w
    w *= total / w.sum()
    for _ in range(iters):
        over  = w > caps
        under = w < floors
        w[over]  = caps[over]
        w[under] = floors[under]
        rem = total - w.sum()
        free = ~(over | under)
        if abs(rem) < 1e-10 or free.sum() == 0: break
        headroom = (caps - w).clip(lower=0.0)
        dist_base = (basis * headroom).where(free, 0.0)
        if dist_base.sum() <= 0:
            eq = headroom.where(free, 0.0)
            if eq.sum() <= 0: break
            w += rem * (eq / eq.sum())
        else:
            w += rem * (dist_base / dist_base.sum())
    if w.sum() != 0:
        w *= total / w.sum()
    return w.clip(lower=floors, upper=caps)

ranked["Weight"] = project_with_pos_caps(w, floors, caps, TARGET_SUM, basis)

# Targets
ranked["Alloc_₹"]       = (ranked["Weight"] * TOTAL_CAPITAL).round(0).astype(int)
ranked["Target_Shares"] = np.floor(ranked["Alloc_₹"] / ranked["Last_Close"]).astype(int)

# Current holdings (settled + T1 + pledged)
try:
    h = pd.DataFrame(kite.holdings())
    if h.empty:
        h = pd.DataFrame(columns=["tradingsymbol","exchange","quantity","t1_quantity","collateral_quantity","last_price"])
    for col in ["quantity","t1_quantity","collateral_quantity","last_price"]:
        if col in h.columns:
            h[col] = pd.to_numeric(h[col], errors="coerce").fillna(0.0)
    h["NormTicker"] = h["tradingsymbol"].map(norm_sym)
    h["actual_qty"] = h.get("quantity", 0.0) + h.get("t1_quantity", 0.0) + h.get("collateral_quantity", 0.0)
    hold_sym = (h.groupby("NormTicker", as_index=False)
                  .agg(actual_qty=("actual_qty","sum"),
                       last_price=("last_price","last")))
except Exception:
    hold_sym = pd.DataFrame(columns=["NormTicker","actual_qty","last_price"])

sym_qty_map = dict(zip(hold_sym["NormTicker"], hold_sym["actual_qty"]))
ranked["Current_Shares"] = ranked["NormTicker"].map(sym_qty_map).fillna(0.0).round(0).astype(int)
ranked["ΔShares"]        = ranked["Target_Shares"] - ranked["Current_Shares"]
ranked["Δ₹"]             = (ranked["ΔShares"] * ranked["Last_Close"]).round(0).astype(int)

# Exit logic: don't sell Buffer if held
sel_core_syms = set(ranked["NormTicker"])
no_sell_syms  = CORE_SYMS | (BUFFER_SYMS & set(hold_sym["NormTicker"]))
held_norm_all = set(hold_sym["NormTicker"])
to_exit = sorted(list(held_norm_all - no_sell_syms))

exit_rows = []
if not hold_sym.empty and len(to_exit):
    extra = hold_sym[hold_sym["NormTicker"].isin(to_exit)].copy()
    for _, r in extra.iterrows():
        q  = float(r.get("actual_qty", 0.0) or 0.0)
        px = float(r.get("last_price", np.nan)) if pd.notna(r.get("last_price", np.nan)) else np.nan
        if q > 0 and pd.notna(px):
            exit_rows.append({
                "Ticker": r["NormTicker"],
                "Final_Rank": np.nan,
                "Last_Close": px,
                "Target_Shares": 0.0,
                "Current_Shares": q,
                "ΔShares": -q,
                "Δ₹": round(-q*px, 0),
            })
exit_df = pd.DataFrame(exit_rows)

# Output tables
alloc_cols = ["Ticker","Final_Rank","Weight","Alloc_₹","Last_Close",
              "Target_Shares","Current_Shares","ΔShares","Δ₹",
              "AnnVol_%_1Y","Beta_1Y","MaxDD"]
alloc_df = (ranked[alloc_cols].sort_values(["Final_Rank"]).reset_index(drop=True))
alloc_df["Weight"] = (alloc_df["Weight"]*100).round(2)

buy_df  = alloc_df[alloc_df["ΔShares"] > 0].copy().sort_values(["Final_Rank"])
sell_df = pd.concat([
            alloc_df[alloc_df["ΔShares"] < 0],
            exit_df[["Ticker","Final_Rank","Last_Close","Target_Shares","Current_Shares","ΔShares","Δ₹"]]
        ], ignore_index=True, sort=False).copy()

print(f"Capital: ₹{TOTAL_CAPITAL:,.0f} | Cash buffer: {CASH_BUFFER_PCT*100:.1f}%")
print(f"Core size: {TARGET_COUNT} | Buffer size: {BUFFER_COUNT}")
display(buy_df.head(20)); display(sell_df.head(20)); display(alloc_df.head(20))

# Save CSVs (optional)
alloc_df.to_csv("allocation_core60.csv", index=False)
buy_df.to_csv("trades_buy_core60.csv", index=False)
sell_df.to_csv("trades_sell_core60.csv", index=False)


Allocator filtered to CORE: 50/81 names
Capital: ₹3,500,000 | Cash buffer: 1.0%
Core size: 50 | Buffer size: 10


/tmp/ipython-input-2571038073.py:152: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sell_df = pd.concat([


,Ticker,Final_Rank,Weight,Alloc_₹,Last_Close,Target_Shares,Current_Shares,ΔShares,Δ₹,AnnVol_%_1Y,Beta_1Y,MaxDD
0,LAURUSLABS,1,6.00,210000,826.00,254,71,183,151158,36.535749,1.180663,-0.186949
1,CARTRADE,2,4.93,172705,2167.20,79,2,77,166874,53.053167,1.354937,-0.233053
2,PAYTM,3,4.79,167594,1057.90,158,74,84,88864,48.545354,1.325107,-0.348369
3,ECLERX,6,4.76,166723,4158.00,40,6,34,141372,39.776809,1.054977,-0.373587
4,NAZARA,7,5.75,201132,1398.80,143,34,109,152469,32.221377,0.766698,-0.211431
5,CHOICEIN,8,5.11,178680,742.90,240,39,201,149323,29.457621,1.054528,-0.174087
6,MAHSCOOTER,9,3.87,135422,14924.00,9,3,6,89544,39.910336,1.200267,-0.277352
7,ENRIN,10,4.55,159079,3077.00,51,8,43,132311,32.906895,NaN,NaN
9,MFSL,13,4.71,164768,1549.40,106,8,98,151841,26.188612,0.644678,-0.233965
10,TFCILTD,14,2.95,103337,286.65,360,104,256,73382,47.993866,0.795239,-0.400589


,Ticker,Final_Rank,Weight,Alloc_₹,Last_Close,Target_Shares,Current_Shares,ΔShares,Δ₹,AnnVol_%_1Y,Beta_1Y,MaxDD
0,POWERINDIA,12,2.82,98550.0,20430.00,4.0,5.0,-1.0,-20430.0,53.861403,1.087117,-0.403060
1,ETERNAL,26,1.34,47019.0,301.50,155.0,180.0,-25.0,-7538.0,40.607754,1.271627,-0.335105
2,BSE,29,0.86,30125.0,2394.00,12.0,50.0,-38.0,-90972.0,61.361290,1.482434,-0.364841
3,WAAREERTL,31,0.78,27243.0,1000.50,27.0,35.0,-8.0,-8004.0,53.501426,1.864853,-0.204848
4,CUMMINSIND,35,0.90,31534.0,3803.00,8.0,11.0,-3.0,-11409.0,31.213907,1.231495,-0.349226
5,VISHNU,41,0.52,18142.0,482.05,37.0,62.0,-25.0,-12051.0,44.176697,1.307645,-0.341172
6,METROPOLIS,42,0.70,24596.0,2043.40,12.0,27.0,-15.0,-30651.0,30.045784,0.796896,-0.349544
7,SHREECEM,43,0.79,27761.0,30515.00,0.0,2.0,-2.0,-61030.0,21.400679,0.861413,-0.155830
8,DIXON,45,0.50,17500.0,15883.00,1.0,6.0,-5.0,-79415.0,41.199297,1.457309,-0.337067
9,SHRIPISTON,46,0.50,17500.0,2380.40,7.0,29.0,-22.0,-52369.0,39.546878,1.113576,-0.284473


,Ticker,Final_Rank,Weight,Alloc_₹,Last_Close,Target_Shares,Current_Shares,ΔShares,Δ₹,AnnVol_%_1Y,Beta_1Y,MaxDD
0,LAURUSLABS,1,6.00,210000,826.00,254,71,183,151158,36.535749,1.180663,-0.186949
1,CARTRADE,2,4.93,172705,2167.20,79,2,77,166874,53.053167,1.354937,-0.233053
2,PAYTM,3,4.79,167594,1057.90,158,74,84,88864,48.545354,1.325107,-0.348369
3,ECLERX,6,4.76,166723,4158.00,40,6,34,141372,39.776809,1.054977,-0.373587
4,NAZARA,7,5.75,201132,1398.80,143,34,109,152469,32.221377,0.766698,-0.211431
5,CHOICEIN,8,5.11,178680,742.90,240,39,201,149323,29.457621,1.054528,-0.174087
6,MAHSCOOTER,9,3.87,135422,14924.00,9,3,6,89544,39.910336,1.200267,-0.277352
7,ENRIN,10,4.55,159079,3077.00,51,8,43,132311,32.906895,NaN,NaN
8,POWERINDIA,12,2.82,98550,20430.00,4,5,-1,-20430,53.861403,1.087117,-0.403060
9,MFSL,13,4.71,164768,1549.40,106,8,98,151841,26.188612,0.644678,-0.233965


In [17]:
# Sheets setup
SPREADSHEET_TITLE = "NSE Allocation DashboardVER2"
today_str = dt.datetime.now().strftime("%Y-%m-%d")
TAB_ALLOCATION = f"Allocation_{today_str}"
TAB_BUY        = f"Buy_{today_str}"
TAB_SELL       = f"Sell_{today_str}"
TAB_META       = f"META_{today_str}"
TAB_BUFFER     = f"Buffer_{today_str}"

# Auth
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open/Create
try:
    sh = gc.open(SPREADSHEET_TITLE)
    print(f"Opened: {SPREADSHEET_TITLE}")
except gspread.SpreadsheetNotFound:
    sh = gc.create(SPREADSHEET_TITLE)
    print(f"Created: {SPREADSHEET_TITLE}")

def upsert_tab(spreadsheet, title, df: pd.DataFrame):
    try:
        ws = spreadsheet.worksheet(title)
    except gspread.WorksheetNotFound:
        ws = spreadsheet.add_worksheet(title=title, rows=max(len(df)+50, 200), cols=max(len(df.columns)+10, 26))
    ws.clear()
    set_with_dataframe(ws, df, include_index=False, include_column_header=True, resize=True)
    return ws

def style_headers(ws):
    n = ws.col_count
    letters = ""
    while n:
        n, r = divmod(n-1, 26)
        letters = chr(65+r) + letters
    header_fmt = CellFormat(textFormat=TextFormat(bold=True, foregroundColor=Color(1,1,1)),
                            backgroundColor=Color(0,0,0))
    format_cell_ranges(ws, [(f"A1:{letters}1", header_fmt)])
    set_frozen(ws, rows=1)

def grid_range_for_df(ws, df, header_rows=1):
    rows = max(len(df), 1) + header_rows
    cols = max(len(df.columns), 1)
    return {"sheetId": ws.id, "startRowIndex": header_rows, "startColumnIndex": 0,
            "endRowIndex": rows, "endColumnIndex": cols}

# Upload tabs
ws_alloc = upsert_tab(sh, TAB_ALLOCATION, alloc_df)
ws_buy   = upsert_tab(sh, TAB_BUY,        buy_df)
ws_sell  = upsert_tab(sh, TAB_SELL,       sell_df)
for ws in (ws_alloc, ws_buy, ws_sell): style_headers(ws)

# Buffer tab (view)
buffer_out = pd.DataFrame({
    "Ticker": BUFFER_TICKERS,
    "Final_Rank": [int(r) for r in buffer["Final_Rank"]] if len(buffer) else [],
    "Final_Score": [round(float(s),1) for s in buffer["Final_Score"]] if len(buffer) else [],
})
buffer_out["Held?"] = [norm_sym(t) in set(hold_sym["NormTicker"]) if "NormTicker" in hold_sym.columns else False
                       for t in buffer_out["Ticker"]] if len(buffer_out) else []
ws_buffer = upsert_tab(sh, TAB_BUFFER, buffer_out)
style_headers(ws_buffer)

# Conditional formatting Buy/Sell
GREEN = {"red":0.86, "green":0.95, "blue":0.87}
RED   = {"red":1.00, "green":0.89, "blue":0.89}
cf_requests = []
if len(buy_df) > 0:
    cf_requests.append({"rule": {"ranges":[grid_range_for_df(ws_buy, buy_df)],
                                 "booleanRule":{"condition":{"type":"NOT_BLANK"},
                                                "format":{"backgroundColor": GREEN}}}, "index":0})
if len(sell_df) > 0:
    cf_requests.append({"rule": {"ranges":[grid_range_for_df(ws_sell, sell_df)],
                                 "booleanRule":{"condition":{"type":"NOT_BLANK"},
                                                "format":{"backgroundColor": RED}}}, "index":0})
if cf_requests:
    sh.batch_update({"requests": [{"addConditionalFormatRule": r} for r in cf_requests]})

# META
meta_df = pd.DataFrame({
    "Field": ["Updated_At", "Sheet_Date", "Rows_Allocation", "Rows_Buy", "Rows_Sell",
              "Core_Size","Buffer_Size","Held∩Buffer"],
    "Value": [dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), today_str,
              len(alloc_df), len(buy_df), len(sell_df),
              len(CORE_TICKERS), len(BUFFER_TICKERS), len(set(BUFFER_SYMS) & set(hold_sym["NormTicker"])) if not hold_sym.empty else 0]
})
ws_meta = upsert_tab(sh, TAB_META, meta_df)
style_headers(ws_meta)

print("✅ Uploaded Allocation / Buy / Sell / Buffer / META")

# Diff vs latest prior Allocation_*
def read_ws_to_df(ws):
    values = ws.get_all_values()
    if not values: return pd.DataFrame()
    hdr, rows = values[0], values[1:]
    return pd.DataFrame(rows, columns=hdr)

all_alloc = []
for _ws in sh.worksheets():
    t = _ws.title
    if t.startswith("Allocation_") and t != TAB_ALLOCATION:
        date_str = t.replace("Allocation_", "")
        try:
            d = dt.datetime.strptime(date_str, "%Y-%m-%d").date()
            all_alloc.append((d, t))
        except ValueError:
            pass

if not all_alloc:
    print("No previous Allocation_* tab found. Skipping Diff.")
else:
    prev_date, prev_title = max(all_alloc, key=lambda x: x[0])
    ws_prev = sh.worksheet(prev_title)
    prev_df = read_ws_to_df(ws_prev)

    keep_cols = ["Ticker","Final_Rank","Weight","Target_Shares","Current_Shares","Last_Close"]
    today_df  = alloc_df[keep_cols].copy()
    prev_df   = prev_df[[c for c in keep_cols if c in prev_df.columns]].copy()

    for c in ["Final_Rank","Weight","Target_Shares","Current_Shares","Last_Close"]:
        if c in today_df.columns: today_df[c] = pd.to_numeric(today_df[c], errors="coerce")
        if c in prev_df.columns:  prev_df[c]  = pd.to_numeric(prev_df[c],  errors="coerce")

    merged = pd.merge(prev_df, today_df, on="Ticker", how="outer", suffixes=("_Prev","_Curr"))
    merged["Status"] = np.select(
        [
            merged["Weight_Prev"].isna() & merged["Weight_Curr"].notna(),
            merged["Weight_Prev"].notna() & merged["Weight_Curr"].isna(),
            (~merged["Weight_Prev"].isna()) & (~merged["Weight_Curr"].isna()) &
            ((merged["Weight_Prev"] != merged["Weight_Curr"]) |
             (merged["Target_Shares_Prev"] != merged["Target_Shares_Curr"]))
        ],
        ["ADDED","REMOVED","CHANGED"], default="NOCHANGE"
    )
    merged["ΔWeight"]        = merged["Weight_Curr"] - merged["Weight_Prev"]
    merged["ΔTarget_Shares"] = merged["Target_Shares_Curr"] - merged["Target_Shares_Prev"]
    merged["ΔRank"]          = merged["Final_Rank_Curr"] - merged["Final_Rank_Prev"]
    merged["Δ₹"]             = pd.to_numeric(merged["ΔTarget_Shares"], errors="coerce").fillna(0) * merged["Last_Close_Curr"].fillna(0)

    diff_cols = [
        "Status","Ticker",
        "Final_Rank_Prev","Final_Rank_Curr","ΔRank",
        "Weight_Prev","Weight_Curr","ΔWeight",
        "Target_Shares_Prev","Target_Shares_Curr","ΔTarget_Shares",
        "Current_Shares_Prev","Current_Shares_Curr",
        "Last_Close_Curr","Δ₹"
    ]
    for c in diff_cols:
        if c not in merged.columns: merged[c] = np.nan

    status_cat = pd.Categorical(merged["Status"], categories=["ADDED","REMOVED","CHANGED","NOCHANGE"], ordered=True)
    merged = merged.assign(_sr=status_cat, _absdw=pd.to_numeric(merged["ΔWeight"], errors="coerce").abs().fillna(0.0))
    merged = merged.sort_values(["_sr","_absdw"], ascending=[True, False]).drop(columns=["_sr","_absdw"])

    round_map = {"Weight_Prev":2,"Weight_Curr":2,"ΔWeight":2,
                 "Target_Shares_Prev":0,"Target_Shares_Curr":0,"ΔTarget_Shares":0,
                 "Final_Rank_Prev":0,"Final_Rank_Curr":0,"ΔRank":0,
                 "Current_Shares_Prev":0,"Current_Shares_Curr":0,
                 "Last_Close_Curr":2,"Δ₹":0}
    for c, r in round_map.items():
        if c in merged.columns: merged[c] = pd.to_numeric(merged[c], errors="coerce").round(r)

    diff_title = f"Diff_{today_str}"
    ws_diff = upsert_tab(sh, diff_title, merged[diff_cols])

    # Summary block (rows 1–3), then style header row 4
    added_cnt   = int((merged["Status"] == "ADDED").sum())
    removed_cnt = int((merged["Status"] == "REMOVED").sum())
    changed_cnt = int((merged["Status"] == "CHANGED").sum())
    total_dwt   = float(pd.to_numeric(merged["ΔWeight"], errors="coerce").fillna(0).sum())
    net_flow    = float(pd.to_numeric(merged["Δ₹"], errors="coerce").fillna(0).sum())
    buys_flow   = float(merged.loc[pd.to_numeric(merged["Δ₹"], errors="coerce") > 0, "Δ₹"].sum())
    sells_flow  = float(-merged.loc[pd.to_numeric(merged["Δ₹"], errors="coerce") < 0, "Δ₹"].sum())

    summary_rows = [
        ["DIFF SUMMARY", "", "", "", "", "", "", ""],
        ["Added","Removed","Changed","Total ΔWeight (%)","Net ₹ Flow","Buys ₹","Sells ₹","Compared To"],
        [added_cnt, removed_cnt, changed_cnt, round(total_dwt,2),
         round(net_flow,0), round(buys_flow,0), round(sells_flow,0), f"vs {prev_title}"],
    ]
    ws_diff.insert_rows([[], [], []], row=1)
    ws_diff.update("A1:H3", summary_rows)

    # Header style row 4
    n = ws_diff.col_count
    letters = ""
    while n:
        n, r = divmod(n-1, 26); letters = chr(65+r) + letters
    hdr_fmt = CellFormat(textFormat=TextFormat(bold=True, foregroundColor=Color(1,1,1)), backgroundColor=Color(0,0,0))
    format_cell_ranges(ws_diff, [(f"A4:{letters}4", hdr_fmt)])
    set_frozen(ws_diff, rows=4)

    # Conditional formatting by Status
    def col_letter(k):
        s="";
        while k: k,r=divmod(k-1,26); s=chr(65+r)+s
        return s
    ncols = len(diff_cols)
    status_col_A1 = col_letter(diff_cols.index("Status")+1)
    rng = {"sheetId": ws_diff.id, "startRowIndex": 4, "startColumnIndex": 0,
           "endRowIndex": 4 + len(merged), "endColumnIndex": ncols}
    GREEN = {"red":0.86, "green":0.95, "blue":0.87}
    RED   = {"red":1.00, "green":0.89, "blue":0.89}
    YELL  = {"red":1.00, "green":0.97, "blue":0.82}
    sh.batch_update({"requests": [
        {"addConditionalFormatRule":{"rule":{"ranges":[rng],"booleanRule":{"condition":{"type":"CUSTOM_FORMULA","values":[{"userEnteredValue": f'=${status_col_A1}5="ADDED"'}]},"format":{"backgroundColor": GREEN}}},"index":0}},
        {"addConditionalFormatRule":{"rule":{"ranges":[rng],"booleanRule":{"condition":{"type":"CUSTOM_FORMULA","values":[{"userEnteredValue": f'=${status_col_A1}5="REMOVED"'}]},"format":{"backgroundColor": RED}}},"index":0}},
        {"addConditionalFormatRule":{"rule":{"ranges":[rng],"booleanRule":{"condition":{"type":"CUSTOM_FORMULA","values":[{"userEnteredValue": f'=${status_col_A1}5="CHANGED"'}]},"format":{"backgroundColor": YELL}}},"index":0}}
    ]})
    print(f"✅ Diff created: {diff_title}")

Opened: NSE Allocation DashboardVER2
✅ Uploaded Allocation / Buy / Sell / Buffer / META
No previous Allocation_* tab found. Skipping Diff.
